In [9]:
pip install alpha_vantage

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from alpha_vantage.timeseries import TimeSeries
# from pprint import pprint
import requests
from datetime import datetime

# 1

In [11]:
class ScriptData:
    def __init__(self):
        self.data = {}
    
    def fetch_intraday_data(self, script):
        api_key = 'TCWNU2ZM9THSTMQ1'
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=5min&apikey={api_key}"
        response = requests.get(url)
        if response.status_code == 200:
            self.data[script] = response.json()
        else:
            raise Exception("Failed to fetch data from Alpha Vantage")
    
    def convert_intraday_data(self, script):
        raw_data = self.data[script]["Time Series (5min)"]
        data = []
        for timestamp, values in raw_data.items():
            data.append([pd.Timestamp(timestamp), float(values["1. open"]), float(values["2. high"]), float(values["3. low"]), float(values["4. close"]), int(values["5. volume"])])
        self.data[script] = pd.DataFrame(data, columns=["timestamp", "open", "high", "low", "close", "volume"])
    
    def __getitem__(self, script):
        return self.data[script]
    
    def __setitem__(self, script, df):
        self.data[script] = df
    
    def __contains__(self, script):
        return script in self.data

In [12]:
script_data = ScriptData()

In [13]:
script_data.fetch_intraday_data('GOOGL')
script_data.convert_intraday_data('GOOGL')

In [14]:
script_data['GOOGL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,94.3500,94.40,94.28,94.4000,13261
1,2023-02-13 19:55:00,94.3500,94.40,94.35,94.3500,6407
2,2023-02-13 19:50:00,94.2800,94.39,94.28,94.3500,2081
3,2023-02-13 19:45:00,94.4200,94.44,94.29,94.2900,2133
4,2023-02-13 19:40:00,94.4700,94.47,94.35,94.3500,917
...,...,...,...,...,...,...
95,2023-02-13 12:05:00,94.2900,94.31,94.22,94.2900,398928
96,2023-02-13 12:00:00,94.2701,94.31,94.24,94.3000,467125
97,2023-02-13 11:55:00,94.2500,94.30,94.12,94.2725,553289
98,2023-02-13 11:50:00,94.3064,94.36,94.23,94.2500,503563


In [15]:
script_data.fetch_intraday_data('AAPL')
script_data.convert_intraday_data('AAPL')

In [16]:
script_data['AAPL']

,timestamp,open,high,low,close,volume
0,2023-02-13 20:00:00,153.660,153.7000,153.60,153.700,7841
1,2023-02-13 19:55:00,153.640,153.6700,153.64,153.660,4691
2,2023-02-13 19:50:00,153.650,153.6500,153.61,153.630,3566
3,2023-02-13 19:45:00,153.650,153.6700,153.65,153.670,2083
4,2023-02-13 19:40:00,153.650,153.6600,153.65,153.660,6946
...,...,...,...,...,...,...
95,2023-02-13 12:05:00,153.440,153.6700,153.41,153.570,477667
96,2023-02-13 12:00:00,153.550,153.5599,153.28,153.420,528881
97,2023-02-13 11:55:00,153.650,153.6900,153.51,153.550,598249
98,2023-02-13 11:50:00,153.510,153.6700,153.45,153.645,614682


In [17]:
'GOOGL' in script_data

True

In [18]:
'AAPL' in script_data

True

In [19]:
'NVDA' in script_data

False

# 2

In [20]:
def indicator1(df, timeperiod = 5):
    close_data = df["close"]
    indicator = close_data.rolling(window=timeperiod).mean()
    return pd.DataFrame({"timestamp": df["timestamp"], "indicator": indicator})

In [21]:
indicator1(script_data['GOOGL'], timeperiod = 5)

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:55:00,NaN
2,2023-02-13 19:50:00,NaN
3,2023-02-13 19:45:00,NaN
4,2023-02-13 19:40:00,94.34800
...,...,...
95,2023-02-13 12:05:00,94.37956
96,2023-02-13 12:00:00,94.34656
97,2023-02-13 11:55:00,94.31106
98,2023-02-13 11:50:00,94.28206


In [22]:
indicator1(script_data['AAPL'], timeperiod = 5)

,timestamp,indicator
0,2023-02-13 20:00:00,NaN
1,2023-02-13 19:55:00,NaN
2,2023-02-13 19:50:00,NaN
3,2023-02-13 19:45:00,NaN
4,2023-02-13 19:40:00,153.66400
...,...,...
95,2023-02-13 12:05:00,153.83488
96,2023-02-13 12:00:00,153.68798
97,2023-02-13 11:55:00,153.60598
98,2023-02-13 11:50:00,153.56898


# 3

In [118]:
import pandas as pd
pd.options.mode.chained_assignment = None

class Strategy:

    def __init__(self, script_name):
        self.script_name = script_name
        self.script_data = ScriptData()
        if script_name not in self.script_data:
            self.script_data.fetch_intraday_data(script_name)
            self.script_data.convert_intraday_data(script_name)
    def get_script_data(self):
        self.df = self.script_data[self.script_name]
    
    def get_signals(self):
        # Fetch intraday historical data
        script_data = ScriptData()        
        # Compute indicator data
        indicator_data = indicator1(self.df)
        # Generate signals DataFrame
        signals = pd.DataFrame({'timestamp': self.df['timestamp'], 'signal': [''] * len(self.df)})
        for i in range(len(self.df)):
          if indicator_data.shift()['indicator'][i] != None:
            if indicator_data.shift()['indicator'][i] < self.df['close'][i]:
              signals['signal'][i] = 'BUY'
            else:
              signals['signal'][i] = 'SELL'
          else:
            signals['signal'][i] = 'No Signal'
        
        # Print signals DataFrame
        print(signals)

In [119]:
strategy = Strategy('NVDA')

In [120]:
strategy.get_script_data()

In [121]:
strategy.get_signals()

             timestamp signal
0  2023-02-13 20:00:00   SELL
1  2023-02-13 19:55:00   SELL
2  2023-02-13 19:50:00   SELL
3  2023-02-13 19:45:00   SELL
4  2023-02-13 19:40:00   SELL
..                 ...    ...
95 2023-02-13 12:05:00    BUY
96 2023-02-13 12:00:00    BUY
97 2023-02-13 11:55:00   SELL
98 2023-02-13 11:50:00    BUY
99 2023-02-13 11:45:00    BUY

[100 rows x 2 columns]


# 4

In [86]:
!pip install mplfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [87]:
!pip install pyalgotrading

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [122]:
import mplfinance as mpf
import pyalgotrading as pat

In [137]:
# import pyalgotrading.plotting as plt

# create a Strategy instance and fetch the required data
# strategy = Strategy('AAPL')

# strategy.get_script_data()
df1 = script_data['GOOGL']
# df = strategy.script_name.convert_intraday_data()

# compute indicator data
indicator_data = indicator1(df1, timeperiod=5)

ap = mpf.make_addplot(df1['timestamp'])

mpf.plot(df1, type='candle', addplot=ap)
# mpf.plot(indicator1(script_data['AAPL'], timeperiod = 5), type='candle')

TypeError: ignored

NameError: ignored